<!-- TABS -->
# Multimodal vector search - Video

<!-- TABS -->
## Configure your production system

:::note
If you would like to use the production features 
of SuperDuperDB, then you should set the relevant 
connections and configurations in a configuration 
file. Otherwise you are welcome to use "development" mode 
to get going with SuperDuperDB quickly.
:::

In [9]:
import os

os.makedirs('.superduperdb', exist_ok=True)
os.environ['SUPERDUPERDB_CONFIG'] = '.superduperdb/config.yaml'

In [10]:
# <tab: MongoDB Community>
CFG = '''
data_backend: mongodb://127.0.0.1:27017/saved_posts
artifact_store: filesystem://../insta_posts_data/artifact_store
'''

In [11]:
with open(os.environ['SUPERDUPERDB_CONFIG'], 'w') as f:
    f.write(CFG)

<!-- TABS -->
## Connect to SuperDuperDB

:::note
Note that this is only relevant if you are running SuperDuperDB in development mode.
Otherwise refer to "Configuring your production system".
:::

In [14]:
# <tab: MongoDB>
from superduperdb import superduper

db = superduper('mongodb://localhost:27017/saved_posts')

2024-Jul-03 02:04:48.30| INFO     | godcreator333| superduperdb.base.build:65   | Data Client is ready. MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000)
2024-Jul-03 02:04:48.30| INFO     | godcreator333| superduperdb.base.build:38   | Connecting to Metadata Client with engine:  MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000)
2024-Jul-03 02:04:48.30| INFO     | godcreator333| superduperdb.base.build:154  | Connecting to compute client: None
2024-Jul-03 02:04:48.30| INFO     | godcreator333| superduperdb.base.datalayer:87   | Building Data Layer


FileNotFoundError: [Errno 2] No such file or directory: '../insta_posts_data/artifact_store/2b075448d752d92c43b9fff29df15ad838e6793c'

<!-- TABS -->
## Get useful sample data

In [ ]:
import os
import requests
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)
mongo_db = client['saved_posts']
collection = mongo_db['posts']

# Directory to save downloaded videos
download_dir = '../insta_posts_data/videos'
os.makedirs(download_dir, exist_ok=True)

# Fetch documents from the collection
posts = collection.find()

# Iterate over the documents and download media if not already downloaded
for post in posts:
    media_link = post.get('media_link')
    if media_link:
        # Get the media file name from the URL
        file_name = os.path.join(download_dir, media_link.split('?')[0].split('/')[-1])
        
        # Check if file already exists
        if not os.path.exists(file_name):
            # Download the media file
            response = requests.get(media_link)
            with open(file_name, 'wb') as file:
                file.write(response.content)
            
            print(f'Downloaded {file_name}')
        else:
            print(f'{file_name} already exists, skipping download.')

# List downloaded files
data = [os.path.join(download_dir, x) for x in os.listdir(download_dir)]
sample_datapoint = data[-1]

# Assuming you need the pil_image import as well
from superduperdb.ext.pillow import pil_image
chunked_model_datatype = pil_image

In [ ]:
datas = [{'x': d} for d in data[:5]]
print("Contents of datas:")
for data in datas:
    print(data)


<!-- TABS -->
## Create datatype

SuperduperDB supports automatic data conversion, so users don’t need to worry about the compatibility of different data formats (`PIL.Image`, `numpy.array`, `pandas.DataFrame`, etc.) with the database.

It also supports custom data conversion methods for transforming data, such as defining the following Datatype.

In [ ]:
# <tab: Video>
from superduperdb import DataType

# Create an instance of the Encoder with the identifier 'video_on_file' and load_hybrid set to False
datatype = DataType(
    identifier='video_on_file',
    encodable='file',
)


<!-- TABS -->
## Setup tables or collections

In [ ]:
from superduperdb.components.table import Table
from superduperdb import Schema
from superduperdb import superduper

db = superduper('mongodb://localhost:27017/saved_posts')
schema = Schema(identifier="schema", fields={"x": datatype})
table_or_collection = Table("documents", schema=schema)
db.apply(table_or_collection)

Inserting data, all fields will be matched with the schema for data conversion.

In [ ]:
import os

print("Current working directory:", os.getcwd())
artifact_store_path = '../insta_posts_data/artifact_store'
print("Artifact store path exists:", os.path.exists(artifact_store_path))

In [ ]:

db['documents'].insert(datas).execute()
select = db['documents'].select()

In [ ]:
from superduperdb.components.table import Table
from superduperdb import Schema
from superduperdb import superduper

db = superduper('mongodb://localhost:27017/saved_posts')
schema = Schema(identifier="schema", fields={"x": datatype})
table_or_collection = Table("documents", schema=schema)
db.apply(table_or_collection)



<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [ ]:
# <tab: Video>
import cv2
import tqdm
from PIL import Image
from superduperdb.ext.pillow import pil_image
from superduperdb import model, Schema
from superduperdb import Listener


@model(
    flatten=True,
    model_update_kwargs={'document_embedded': False},
)
    
def chunker(video_file):
    # Set the sampling frequency for frames
    sample_freq = 10
    
    # Open the video file using OpenCV
    cap = cv2.VideoCapture(video_file)
    
    # Initialize variables
    frame_count = 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    extracted_frames = []
    progress = tqdm.tqdm()

    # Iterate through video frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Get the current timestamp based on frame count and FPS
        current_timestamp = frame_count // fps
        
        # Sample frames based on the specified frequency
        if frame_count % sample_freq == 0:
            extracted_frames.append({
                'image': Image.fromarray(frame[:,:,::-1]),  # Convert BGR to RGB
                'current_timestamp': current_timestamp,
            })
        frame_count += 1
        progress.update(1)
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    
    # Return the list of extracted frames
    return extracted_frames

Now we apply this chunker to the data by wrapping the chunker in `Listener`:

In [ ]:
from superduperdb import Listener

upstream_listener = Listener(
    model=chunker,
    select=select,
    key='x',
    uuid="chunk",
)

db.apply(upstream_listener)

## Build multimodal embedding models

We define the output data type of a model as a vector for vector transformation.

In [ ]:
# <tab: MongoDB>
from superduperdb.components.vector_index import vector
output_datatpye = vector(shape=(1024,))

Then define two models, one for text embedding and one for image embedding.

In [ ]:
# <tab: Text-Image>
import clip
from superduperdb import vector
from superduperdb.ext.torch import TorchModel

# Load the CLIP model and obtain the preprocessing function
model, preprocess = clip.load("ViT-B/32", device='cpu')

# Create a TorchModel for text encoding
compatible_model = TorchModel(
    identifier='clip_text', # Unique identifier for the model
    object=model, # CLIP model
    preprocess=lambda x: clip.tokenize(x)[0],  # Model input preprocessing using CLIP 
    postprocess=lambda x: x.tolist(), # Convert the model output to a list
    datatype=output_datatpye,  # Vector encoder with shape (1024,)
    forward_method='encode_text', # Use the 'encode_text' method for forward pass 
)

# Create a TorchModel for visual encoding
model = TorchModel(
    identifier='clip_image',  # Unique identifier for the model
    object=model.visual,  # Visual part of the CLIP model    
    preprocess=preprocess, # Visual preprocessing using CLIP
    postprocess=lambda x: x.tolist(), # Convert the output to a list 
    datatype=output_datatpye, # Vector encoder with shape (1024,)
)

Because we use multimodal models, we define different keys to specify which model to use for embedding calculations in the vector_index.

In [ ]:
compatible_key = 'text' # we use text key for text embedding
indexing_key = upstream_listener.outputs_key + '.image' # we use indexing_key for image embedding, use the image field of the result
select = upstream_listener.outputs_select

## Create vector-index

In [ ]:
vector_index_name = 'my-vector-index'

In [ ]:
# <tab: 2-Modalities>
from superduperdb import VectorIndex, Listener

jobs, _ = db.add(
    VectorIndex(
        vector_index_name,
        indexing_listener=Listener(
            key=indexing_key,      # the `Document` key `model` should ingest to create embedding
            select=select,       # a `Select` query telling which data to search over
            model=model,         # a `_Predictor` how to convert data to embeddings
        ),
        compatible_listener=Listener(
            key=compatible_key,      # the `Document` key `model` should ingest to create embedding
            model=compatible_model,         # a `_Predictor` how to convert data to embeddings
            active=False,
            select=None,
        )
    )
)

In [ ]:
query_table_or_collection = select.table_or_collection

## Perform a vector search

We can perform the vector searches using text description:

In [ ]:
# <tab: Text>
from superduperdb import Document
item = Document({compatible_key: "monkeys"})

Once we have this search target, we can execute a search as follows.

In [ ]:
select = query_table_or_collection.like(item, vector_index=vector_index_name, n=5).select()
results = list(db.execute(select))

## Visualize Results

In [ ]:
from IPython.display import display
for result in results:
    display(Document(result.unpack())[indexing_key])

## Check the system stays updated

You can add new data; once the data is added, all related models will perform calculations according to the underlying constructed model and listener, simultaneously updating the vector index to ensure that each query uses the latest data.

In [ ]:
new_datas = [{'x': data[-1]}]
ids = db['documents'].insert(new_datas).execute()